In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
# import imutils
from tqdm import tqdm
from abc import ABC, abstractmethod
from nptyping import NDArray
from typing import Any, Tuple, List, Union, Optional
from dataclasses import dataclass
from enum import Enum

# sys.path.append(os.path.join("..", ".."))


In [ ]:
ImageType = Union[NDArray[(Any, Any), np.uint8], NDArray[(Any, Any, 2), np.uint8], NDArray[(Any, Any, 3), np.uint8], NDArray[(Any, Any, 4), np.uint8]]


class ImageFormat(Enum):
    """Image format"""

    BLACK_AND_WHITE = "BLACK_AND_WHITE"
    BLACK_AND_WHITE_WITH_TRANSPARENCY = "BLACK_AND_WHITE_WITH_TRANSPARENCY"
    COLORED = "COLORED"
    COLORED_WITH_TRANSPARENCY = "COLORED_WITH_TRANSPARENCY"

def format_image(image: ImageType, to_format) -> Tuple[ImageType, Optional[NDArray[(Any, Any), np.uint8]]]:
    """To format a nD image into a mD one"""
    if to_format == ImageFormat.BLACK_AND_WHITE:
        if image.ndim == 2:
            return image,
        if image.ndim == 3 and image.shape[-1] == 2:
            mask = image[:, :, 1]
            image = image[:, :, 0]
            return image, mask
        if image.ndim == 3 and image.shape[-1] == 3:
            return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY),
        if image.ndim == 3 and image.shape[-1] == 4:
            mask = image[:, :, 3],
            image = cv2.cvtColor(image[:, :, :3], cv2.COLOR_RGB2GRAY)
            return image, mask
    if to_format == ImageFormat.BLACK_AND_WHITE_WITH_TRANSPARENCY:
        if image.ndim == 2:
            image_with_transparency = np.zeros((image.shape[0], image.shape[1], 2), dtype=np.uint8)
            mask = 255 * np.ones((image.shape[0], image.shape[1]), dtype=np.uint8)
            image_with_transparency[:, :, 0] = image
            image_with_transparency[:, :, 1] = mask
            return image, mask
        if image.ndim == 3 and image.shape[-1] == 2:
            return image, image[:, :, 3]
        if image.ndim == 3 and image.shape[-1] == 3:
            image_with_transparency = np.zeros((image.shape[0], image.shape[1], 2), dtype=np.uint8)
            mask = 255 * np.ones((image.shape[0], image.shape[1]), dtype=np.uint8)
            image_with_transparency[:, :, 0] = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            image_with_transparency[:, :, 1] = mask
            return image_with_transparency, mask
        if image.ndim == 3 and image.shape[-1] == 4:
            image_with_transparency = np.zeros((image.shape[0], image.shape[1], 2), dtype=np.uint8)
            mask = image[:, :, 3],
            image_with_transparency[:, :, 0] = cv2.cvtColor(image[:, :, :3], cv2.COLOR_RGB2GRAY)
            image_with_transparency[:, :, 1] = mask
            return image, mask
    if to_format == ImageFormat.COLORED:
        if image.ndim == 2:
            return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB),
        if image.ndim == 3 and image.shape[-1] == 2:
            mask = image[:, :, 1]
            image = image[:, :, 0]
            return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB), mask
        if image.ndim == 3 and image.shape[-1] == 3:
            return image,
        if image.ndim == 3 and image.shape[-1] == 4:
            mask = image[:, :, 3],
            image = image[:, :, :3]
            return image, mask
    if to_format == ImageFormat.COLORED_WITH_TRANSPARENCY:
        if image.ndim == 2:
            image_with_transparency = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)
            mask = 255 * np.ones((image.shape[0], image.shape[1]), dtype=np.uint8)
            image_with_transparency[:, :, :3] = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            image_with_transparency[:, :, 3] = mask
            return image_with_transparency, mask
        if image.ndim == 3 and image.shape[-1] == 2:
            image_with_transparency = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)
            mask = image[:, :, 1]
            image_with_transparency[:, :, :3] = cv2.cvtColor(image[:, :, 0], cv2.COLOR_GRAY2RGB)
            image_with_transparency[:, :, 3] = mask
            return image_with_transparency, mask
        if image.ndim == 3 and image.shape[-1] == 3:
            image_with_transparency = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)
            mask = 255 * np.ones((image.shape[0], image.shape[1]), dtype=np.uint8)
            image_with_transparency[:, :, :3] = image
            image_with_transparency[:, :, 3] = mask
            return image_with_transparency, mask
        if image.ndim == 3 and image.shape[-1] == 4:
            return image, image[:, :, 3]

In [ ]:
class Transformer(ABC):
    """Generic image transformation"""

    @abstractmethod
    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""

class Combiner(ABC):
    """Generic image combiner"""

    @abstractmethod
    def combine(self, input_img1: ImageType, input_img2: ImageType) -> ImageType:
        """Combine two images"""

In [ ]:
class GaussianBlurTransformer(Transformer):
    """Apply gaussian blur to transformer"""

    def __init__(self, kernel: Tuple[int, int] = (5, 5), stdev: int = 0):
        """Initialize the blur parameters"""
        self.kernel = kernel
        self.stdev = stdev

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        return cv2.GaussianBlur(input_img, self.kernel, self.stdev)

class MedianBlurTransformer(Transformer):
    """Apply median blur to transformer"""

    def __init__(self, ksize: int = 5):
        """Initialize the blur parameters"""
        self.ksize = ksize

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        return cv2.medianBlur(input_img, self.ksize)

class BilateralBlurTransformer(Transformer):
    """Apply bilateral blur to transformer"""

    def __init__(self, sigma_color: int = 5, sigma_space: int = 80, border_type: int = 80):
        """Initialize the blur parameters"""
        self.sigma_color = sigma_color
        self.sigma_space = sigma_space
        self.border_type = border_type

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        return cv2.bilateralFilter(input_img, self.sigma_color, self.sigma_space, self.border_type)

class CannyEdgeTransformer(Transformer):
    """Apply canny edge detection"""

    def __init__(self, th_min: int = 30, th_max: int = 150):
        """Initialize the canny edge detector"""
        self.th_min = th_min
        self.th_max = th_max

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        return cv2.Canny(input_img, self.th_min, self.th_max)

class DilateErodeTransformer(Transformer):
    """Delete noise by dilating then eroding"""

    def __init__(self, kernel: Tuple[int, int] = (3, 3)):
        """Initialize the affine transformation"""
        self.kernel = np.ones(kernel, np.uint8)

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        dilated = cv2.dilate(input_img, self.kernel, iterations=1)
        return cv2.erode(dilated, self.kernel, iterations=1)

class ContourTransformer(Transformer):
    """Draw contours of an image"""
    def __init__(self, mode: int = cv2.RETR_TREE, method: int = cv2.CHAIN_APPROX_SIMPLE, edge_color: int = 0, edge_thickness: int = 1):
        """Initialize the contour transformer"""
        self.mode = mode
        self.method = method
        self.edge_color = edge_color
        self.edge_thickness = edge_thickness

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        contours, _ = cv2.findContours(input_img, self.mode, self.method)
        # sort contours by area
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        # create mask for drawing contours
        mask = 255 * np.ones(input_img.shape, dtype=np.uint8)
        # draw contours on mask
        cv2.drawContours(mask, contours, contourIdx=-1, color=self.edge_color, thickness=self.edge_thickness)

        # format
        image_with_mask = np.zeros((input_img.shape[0], input_img.shape[1], 2), dtype=np.uint8)
        image_with_mask[:, :, 0] = mask
        image_with_mask[:, :, 1] = cv2.bitwise_not(mask)
        return image_with_mask

class AdaptiveThresholdContour(Transformer):
    """Draw contours thanks to adaptive threshold"""

    def __init__(self, line_size: int = 3, blur_value: int = 3):
        """Initialize the adaptive threshold contour"""
        self.line_size = line_size
        self.blur_value = blur_value

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        edges = cv2.adaptiveThreshold(input_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, self.line_size, self.blur_value)
        image_with_mask = np.zeros((input_img.shape[0], input_img.shape[1], 2), dtype=np.uint8)
        image_with_mask[:, :, 0] = edges
        image_with_mask[:, :, 1] = cv2.bitwise_not(edges)
        return image_with_mask

class ColorTransformer(Transformer):
    """To transform an image of a certain type to another"""
    
    def __init__(self, to_format: ImageFormat, return_mask: bool = False):
        """Initialize the image converter"""
        self.to_format = to_format
        self.return_mask = return_mask

    def __call__(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        return self.transform(input_img)

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        idx = 1 if self.return_mask else 0
        return format_image(input_img, self.to_format)[idx]

class KmeansQuantizationTransformer(Transformer):
    """Quantize an image to a certain number of colors using k-means"""

    def __init__(self, n_colors: int):
        """Initialize the color quantization transformer"""
        self.n_colors = n_colors

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        data = np.float32(input_img).reshape((-1, 3))

        # Determine criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)

        # Implementing K-Means
        _, label, center = cv2.kmeans(data, self.n_colors, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        center = np.uint8(center)
        result = center[label.flatten()]
        result = result.reshape(input_img.shape)
        return result

class BinsQuantizationTransformer(Transformer):
    """Quantize an image to a certain number of colors using bins"""

    def __init__(self, n_colors: int):
        """Initialize the color quantization transformer"""
        self.bins = 256//n_colors

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        return input_img // self.bins * self.bins + 128 // self.bins

class HistogramEqualizationTransformer(Transformer):
    """Equalize the histogram of an image"""

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        return cv2.equalizeHist(input_img)

class HistogramMatchingTransformer(Transformer):
    """Matches an images histogram with a given one"""

    def __init__(self, histogram_path: str = "reversed_histo.json"):
        """Initialize the histogram matching transformer"""
        with open(histogram_path, "r", encoding="utf-8") as f:
            self.reverse_hists = json.load(f)

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        hsv = np.asarray(cv2.cvtColor(input_img, cv2.COLOR_RGB2HSV))
        direct_hists = {
            "hue": cv2.calcHist([hsv[:,:,0]],[0],None,[256],[0,256]).flatten(),
            "saturation": cv2.calcHist([hsv[:,:,1]],[0],None,[256],[0,256]).flatten(),
            "value": cv2.calcHist([hsv[:,:,2]],[0],None,[256],[0,256]).flatten()
        }
        final_hists = {}
        for k, hist in direct_hists.items():
            hists_cum = np.cumsum(hist)
            hists_norm = (hists_cum/hists_cum[-1]*(len(hists_cum)-1)).astype(np.uint8)
            # final_hists[k] = np.array([self.reverse_hists[k][hists_norm[i]] for i in range(len(hists_norm))])
            final_hists[k] = np.array(self.reverse_hists[k])
        final_img = np.zeros(hsv.shape)
        # final_img[:,:,0] = cv2.LUT(hsv[:,:,0], final_hists["hue"])
        final_img[:,:,0] = hsv[:,:,0]
        final_img[:,:,1] = cv2.LUT(hsv[:,:,1], final_hists["saturation"])
        final_img[:,:,2] = cv2.LUT(hsv[:,:,2], final_hists["value"])
        return cv2.cvtColor(final_img.astype(np.uint8), cv2.COLOR_HSV2RGB)

class HSVAffineTransformer(Transformer):
    """Affine hsv transformation"""

    def __init__(self, h_a: float = 1, h_b: float = 0, s_a: float = 1, s_b: float = 0, v_a: float = 1, v_b: float = 0):
        """Initialize the affine transformation (a*x + b)"""
        self.h_a = h_a
        self.h_b = h_b
        self.s_a = s_a
        self.s_b = s_b
        self.v_a = v_a
        self.v_b = v_b

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        hsv = cv2.cvtColor(input_img, cv2.COLOR_RGB2HSV)
        hsv[:, :, 0] = (hsv[:, :, 0] * self.h_a + self.h_b) % 256
        hsv[:, :, 1] = np.clip(hsv[:, :, 1] * self.s_a + self.s_b, 0, 255)
        hsv[:, :, 2] = np.clip(hsv[:, :, 2] * self.v_a + self.v_b, 0, 255)
        return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

class SpecificColorTransformer(Transformer):
    """Modifines specicied colors."""

    def __init__(self, color_params: List[Tuple[List[int], float, float]]):
        """Initialize the specific color transformer"""
        self.color_params = color_params

In [ ]:
class MaskCombiner(Combiner):
    """Combine an image with a mask"""

    def __init__(self, opaqueness: float = 1):
        """Initialize the mask combiner"""
        self.opaqueness = opaqueness

    def combine(self, image1: ImageType, image2_with_mask: ImageType) -> ImageType:
        """Combine two images"""
        # return cv2.bitwise_and(input_img, input_img, mask=mask)

        mask = image2_with_mask[:, :, -1]
        image2 = image2_with_mask[:, :, :-1]

        # get first masked value (foreground)
        fg = cv2.bitwise_or(image2, image2, mask=mask)

        # get second masked value (background) mask must be inverted
        mask = cv2.bitwise_not(mask)
        bk = cv2.bitwise_or(image1, image1, mask=mask)

        # combine foreground+background
        combined = cv2.bitwise_or(fg, bk)

        # Use opaqueness
        return (self.opaqueness * combined + (1 - self.opaqueness) * image1).astype(np.uint8)

In [ ]:
@dataclass
class TransformerChain:
    """Defines a transformer chain"""
    input_name: str
    output_name: str
    transformers: List[Transformer]

@dataclass
class CombinerType:
    """Defines a combiner chain"""
    input_name1: str
    input_name2: str
    output_name: str
    combiner: Combiner

class PipelineTransformer(Transformer):
    def __init__(self, actuators: List[Union[TransformerChain, CombinerType]]):
        """Initialize the global combiner"""
        self.actuators = actuators

    def transform(self, input_img: ImageType) -> ImageType:
        """Applies transform to an image"""
        images = {"input": input_img}
        for actuator in self.actuators:
            if isinstance(actuator, TransformerChain):
                image = actuator.transformers[0].transform(images[actuator.input_name])
                for transformer in actuator.transformers[1:]:
                    image = transformer.transform(image)
                images[actuator.output_name] = image
            elif isinstance(actuator, CombinerType):
                images[actuator.output_name] = actuator.combiner.combine(images[actuator.input_name1], images[actuator.input_name2])
        return images["output"]

In [ ]:
def read_image(image_path: str) -> np.ndarray:
    """Read image from path"""
    image = cv2.imread(image_path)
    assert image is not None, "Image is None"
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [ ]:
transformer_quantized_colors = TransformerChain(
    input_name="input",
    output_name="output",
    transformers=[BinsQuantizationTransformer(n_colors=10), GaussianBlurTransformer(kernel=(7,7)), HistogramMatchingTransformer()]
)
transformer_chain_contours = TransformerChain(
    input_name="input",
    output_name="contours",
    # transformers=[ColorTransformer(to_format=ImageFormat.BLACK_AND_WHITE), BlurTransformer(kernel=(7,7)), CannyEdgeTransformer(), DilateErodeTransformer(), BlurTransformer(), ContourTransformer(), ColorTransformer(to_format=ImageFormat.COLORED_WITH_TRANSPARENCY)]
    transformers=[ColorTransformer(to_format=ImageFormat.BLACK_AND_WHITE), GaussianBlurTransformer(kernel=(7,7)), AdaptiveThresholdContour(), ColorTransformer(to_format=ImageFormat.COLORED_WITH_TRANSPARENCY)]
)
combiner_cartoon = CombinerType(
    input_name1="quantized_colors",
    input_name2="contours",
    output_name="output",
    combiner=MaskCombiner(opaqueness=1)
)
# pipeline_cartoon = PipelineTransformer([transformer_quantized_colors, transformer_chain_contours, combiner_cartoon])
pipeline_cartoon = PipelineTransformer([transformer_quantized_colors])

In [ ]:
paths = ["data/flickr/Images/667626_18933d713e.jpg", "data/flickr/Images/17273391_55cfc7d3d4.jpg", "data/flickr/Images/49553964_cee950f3ba.jpg", "data/flickr/Images/112243673_fd68255217.jpg", "data/flickr/Images/357725852_6f55cb9abc.jpg"]

for path in paths:
    image = read_image(path)
    plt.imshow(image)
    plt.show()
    cartoon = pipeline_cartoon.transform(image)
    plt.imshow(cartoon)
    plt.show()

In [ ]:

# cartoonize image with cv2
def cartoonize_image(image):
    """Cartoonize image with cv2"""
    # convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # apply gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    # apply Canny edge detection
    edges = cv2.Canny(blur, 30, 150)
    # apply dilation and erosion to remove some noise
    kernel = np.ones((3, 3), np.uint8)
    dilation = cv2.dilate(edges, kernel, iterations=1)
    erosion = cv2.erode(dilation, kernel, iterations=1)
    # apply blur to smooth out edges
    blur = cv2.GaussianBlur(erosion, (5, 5), 0)
    # find contours
    contours, _ = cv2.findContours(blur, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # sort contours by area
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    # create mask for drawing contours
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    # draw contours on mask
    cv2.drawContours(mask, contours, -1, (255, 255, 255), -1)
    # apply mask to image
    result = cv2.bitwise_and(image, image, mask=mask)
    return result


In [ ]:
# def load_df_from_csv(csv_file):
#     """Load dataframe from csv file"""
#     df = pd.read_csv(csv_file)
#     return df


In [ ]:
# df_train_cartoons = load_df_from_csv("../../data/cartoons_train.csv")
# df_train_cartoons.head()

In [ ]:
# df_train_images = load_df_from_csv("../../data/images_train.csv")
# df_train_images.head()

In [ ]:
# def read_image(image_path: str) -> np.ndarray:
#     """Read image from path"""
#     image = cv2.imread(os.path.join("..", "..", image_path))
#     assert image is not None, "Image is None"
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     return image

# def compute_image_histogram(image: np.ndarray) -> np.ndarray:
#     """Compute image histogram"""
#     hist = cv2.calcHist([image], channels=[0, 1, 2], mask=None, histSize=[8, 8, 8], ranges=[0, 256, 0, 256, 0, 256])
#     hist = cv2.normalize(hist, hist).flatten()
#     return hist

# def plot_image_and_its_histogram(image: np.ndarray) -> np.ndarray:
#     """Plot image histogram"""
#     plt.figure(figsize=(10, 10))
#     plt.subplot(1, 2, 1)
#     plt.imshow(image)
#     plt.subplot(1, 2, 2)
#     plt.plot(compute_image_histogram(image), color="blue", linewidth=2, linestyle="-", label="histogram")
#     plt.show()

In [ ]:
# # shuffle dataframe
# df_train_cartoons = df_train_cartoons.sample(frac=1).reset_index(drop=True)
# df_train_images = df_train_images.sample(frac=1).reset_index()

In [ ]:
# for i in range(10):
#     plot_image_and_its_histogram(read_image(df_train_cartoons.iloc[i]["path"]))

In [ ]:
# for i in range(10):
#     plot_image_and_its_histogram(read_image(df_train_images.iloc[i]["path"]))

In [ ]:
def compute_color_histograms(image: np.ndarray):
    """Compute color histograms"""
    hist = cv2.calcHist([image], channels=[0, 1, 2], mask=None, histSize=[8, 8, 8], ranges=[0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def plot_color_histogram_and_its_image(image: np.ndarray):
    """Plot color histogram and its image"""
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.subplot(1, 2, 2)
    plt.plot(compute_color_histograms(image), color="blue", linewidth=2, linestyle="-", label="histogram")
    plt.show()

for i in range(10):
    plot_color_histogram_and_its_image(read_image(df_train_cartoons.iloc[i]["path"]))

In [ ]:
def is_cartoon(image: np.ndarray, threshold: float = 0.98) -> bool:
    """Check if image is cartoon"""
    image = cv2.resize(image, (256, 256))

    color_blurred = cv2.bilateralFilter(image, 6, 250, 250)

    # compare the colors from the original image to blurred one.
    diffs = []
    for k, _ in enumerate(('b', 'r', 'g')):
        # print(f"Comparing histogram for color {color}")
        real_histogram = cv2.calcHist(image, [k], None, [256], [0, 256])
        color_histogram = cv2.calcHist(color_blurred, [k], None, [256], [0, 256])
        diffs.append(cv2.compareHist(real_histogram, color_histogram, cv2.HISTCMP_CORREL))

    return sum(diffs) / 3 > threshold

def is_cartoon_color_count(image: np.ndarray, threshold: float = 0.3) -> bool:
    image = cv2.resize(image, (1024, 1024))
    # Find count of each color
    color_count = {}
    for row in image:
        for item in row:
            value = tuple(item)
            if value not in color_count:
                color_count[value] = 1
            else:
                color_count[value] += 1

    # Identify the percent of the image that uses the top 512 colors
    most_common_colors = sum([x[1] for x in sorted(color_count.items(), key=lambda pair: pair[1], reverse=True)[:512]])
    return (most_common_colors / (1024 * 1024)) > threshold

In [ ]:
## with first method

def do_grid_search_on_threshold(method: callable, thresholds: np.ndarray):
    """Do grid search on threshold"""
    threshold_values = np.arange(0.1, 1.0, 0.1)
    best_f1_score = 0
    best_threshold = 0
    for threshold in threshold_values:
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for i in range(100):
            image = read_image(df_train_cartoons.iloc[i]["path"])
            if method(image, threshold):
                tp += 1
            else:
                fn += 1
        for i in range(100):
            image = read_image(df_train_images.iloc[i]["path"])
            if method(image, threshold):
                fp += 1
            else:
                tn += 1
        f1_score = 2 * tp / (2 * tp + fp + fn)
        if f1_score > best_f1_score:
            best_f1_score = f1_score
            best_threshold = threshold
    print(f"Best threshold: {best_threshold}")
    print(f"Best f1 score: {best_f1_score}")




In [ ]:
## compare the two methods

print(do_grid_search_on_threshold(is_cartoon, np.arange(0.1, 1.0, 0.1)))


In [ ]:
print(do_grid_search_on_threshold(is_cartoon_color_count, np.array([0.3])))

In [ ]:
one_test_img = read_image(df_train_images.iloc[0]["path"])

def show_image(image: np.ndarray):
    """Show image"""
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.show()

show_image(one_test_img)

In [ ]:
# Apply some Gaussian blur on the image
img_gb = cv2.GaussianBlur(one_test_img, (7, 7) ,0)
# Apply some Median blur on the image
img_mb = cv2.medianBlur(img_gb, 5)
# Apply a bilateral filer on the image
img_bf = cv2.bilateralFilter(img_mb, 5, 80, 80)

show_image(img_bf)

In [ ]:
img_lp_im = cv2.Laplacian(one_test_img, cv2.CV_8U, ksize=5)
img_lp_gb = cv2.Laplacian(img_gb, cv2.CV_8U, ksize=5)
img_lp_mb = cv2.Laplacian(img_mb, cv2.CV_8U, ksize=5)
img_lp_al = cv2.Laplacian(img_bf, cv2.CV_8U, ksize=5)

show_image(img_lp_im)

In [ ]:
# Convert the image to greyscale (1D)
img_lp_im_grey = cv2.cvtColor(img_lp_im, cv2.COLOR_BGR2GRAY)
img_lp_gb_grey = cv2.cvtColor(img_lp_gb, cv2.COLOR_BGR2GRAY)
img_lp_mb_grey = cv2.cvtColor(img_lp_mb, cv2.COLOR_BGR2GRAY)
img_lp_al_grey = cv2.cvtColor(img_lp_al, cv2.COLOR_BGR2GRAY)

show_image(img_lp_im_grey)

In [ ]:
# Manual image thresholding
_, EdgeImage = cv2.threshold(one_test_img, 127, 255, cv2.THRESH_BINARY)

In [ ]:
# Remove some additional noise
blur_im = cv2.GaussianBlur(img_lp_im_grey, (5, 5), 0)
blur_gb = cv2.GaussianBlur(img_lp_gb_grey, (5, 5), 0)
blur_mb = cv2.GaussianBlur(img_lp_mb_grey, (5, 5), 0)
blur_al = cv2.GaussianBlur(img_lp_al_grey, (5, 5), 0)
# Apply a threshold (Otsu)
_, tresh_im = cv2.threshold(blur_im, 245, 255,cv2.THRESH_BINARY +  cv2.THRESH_OTSU)
_, tresh_gb = cv2.threshold(blur_gb, 245, 255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
_, tresh_mb = cv2.threshold(blur_mb, 245, 255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
_, tresh_al = cv2.threshold(blur_al, 245, 255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)

In [ ]:
# Invert the black and the white
inverted_original = cv2.subtract(255, tresh_im)
inverted_GaussianBlur = cv2.subtract(255, tresh_gb)
inverted_MedianBlur = cv2.subtract(255, tresh_mb)
inverted_Bilateral = cv2.subtract(255, tresh_al)

show_image(inverted_original)

In [ ]:
# Reshape the image
img_reshaped = one_test_img.reshape((-1,3))
# convert to np.float32
img_reshaped = np.float32(img_reshaped)
# Set the Kmeans criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
# Set the amount of K (colors)
K = 8
# Apply Kmeans
_, label, center = cv2.kmeans(img_reshaped, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
# Covert it back to np.int8
center = np.uint8(center)
res = center[label.flatten()]
# Reshape it back to an image
img_Kmeans = res.reshape((one_test_img.shape))

In [ ]:
# Reduce the colors of the original image
div = 64
img_bins = one_test_img // div * div + div // 2

In [ ]:
# Convert the mask image back to color 
inverted_Bilateral = cv2.cvtColor(inverted_Bilateral, cv2.COLOR_GRAY2RGB)
# Combine the edge image and the binned image
cartoon_Bilateral = cv2.bitwise_and(inverted_Bilateral, img_bins)
# Save the image
cv2.imwrite('CartoonImage.png', cartoon_Bilateral)
